In [128]:
from bs4 import BeautifulSoup
import requests
import html5lib
import sqlite3
import uuid
import re
import json

In [35]:
connection = sqlite3.connect('MovieData.db')
cur = connection.cursor()

In [36]:
table1 = 'CREATE TABLE IF NOT EXISTS MovieIndex(Id, Name, Url)'
cur.execute(table1)

In [37]:
baseurl = 'https://tamilpaatu.com'
requesturl = baseurl + '/movie-index'

In [38]:
def getsoup(url):
    headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    source = requests.get(url, headers= headers)
    return BeautifulSoup(source.content, 'html5lib')

In [39]:
movieindexpage = getsoup(requesturl)
tagDiv = movieindexpage.find('div', class_='tagdiv')
movieIndex = tagDiv.find_all('a')
for el in movieIndex:
    indexname = el.text
    url = baseurl + el.get('href')
    indexquery = 'SELECT 1 FROM MovieIndex WHERE Url = ?'
    indexres = cur.execute(indexquery, [url])
    if indexres.fetchone() is None:
        cur.execute('INSERT INTO MovieIndex VALUES (?, ?, ?)', [str(uuid.uuid4()), indexname, url])

In [40]:
connection.commit()

In [24]:
table2 = 'CREATE TABLE IF NOT EXISTS MoviePage(Id, Name, Url, WebpSrc, JpegSrc)'
cur.execute(table2)

In [58]:
song_pages = []
retrive_pages = []
def page_data_parse(href):
    songpage = getsoup(href)
    songscontainer = songpage.find('div', class_='botlist')
    if songscontainer is None:
        if href not in retrive_pages:
            retrive_pages.append(href)
        return
    songs = songscontainer.find_all('div', class_='a-i')
    for song in songs:
        el = song.find('a')
        song_href = baseurl + el.get('href')
        picture = el.find('picture')
        webp_src = baseurl + picture.find_all('source')[0].get('srcset')
        jpeg_src = baseurl + picture.find_all('source')[1].get('srcset')
        info = el.find('div', class_='info')
        title = info.find('h2').text
        
        song_query = 'SELECT 1 FROM MoviePage WHERE Url = ?'
        song_res = cur.execute(song_query, [song_href])
        if song_res.fetchone() is None:
            song_insert_query = 'INSERT INTO MoviePage VALUES (?, ?, ?, ?, ?)'
            cur.execute(song_insert_query, [str(uuid.uuid4()), title, song_href, webp_src, jpeg_src])
            connection.commit()
            
        if song_href not in song_pages:
            song_pages.append(song.find('a').get('href'))

    pagination_container = songscontainer.find('nav', class_='pagination')
    next_page_container = pagination_container.find('span', class_='next')
    next_page = next_page_container.find('a')
    
    if href in retrive_pages:
        retrive_pages.remove(href)

    if next_page is None:
        return
    next_page_href = next_page.get('href')
    page_data_parse(baseurl + next_page_href)

In [59]:
movie_index_query = 'SELECT Url FROM MovieIndex'
movie_index_res = cur.execute(movie_index_query)
movieIndexHref = movie_index_res.fetchall()
for moviepage in movieIndexHref:    
    page_data_parse(moviepage[0])

In [60]:
while 1:
    for moviepage in retrive_pages:
        page_data_parse(moviepage)
    if len(retrive_pages) == 0:
        break

In [151]:
def get_detail(uri, title):
    songdetail_soap = getsoup(uri)
    article = songdetail_soap.find('article', class_='post-content')
    if article is None:
        pass
    movie_information_ele = article.find('fieldset', id='movie-handle')
    movie_information = movie_information_ele.text.strip()
    information_collection = movie_information.split('\n')
    #Casts
    starring_text = list(filter(lambda x: 'Starring' in x, information_collection))[0].strip().split(':')[1]
    casts = [cast.strip() for cast in re.split(r',|&', starring_text)]

    #Music
    music_text = list(filter(lambda x: 'Music' in x, information_collection))[0].strip().split(':')[1]
    music_directors = [director.strip() for director in re.split(r',|&', music_text)]

    #Director
    director_text = list(filter(lambda x: 'Director' in x, information_collection))[0].strip().split(':')[1]
    directors = [director.strip() for director in re.split(r',|&', director_text)]

    #Lyrics 
    lyrics_text = list(filter(lambda x: 'Lyrics' in x, information_collection))[0].strip().split(':')[1]
    lyric_writers = [writer.strip() for writer in re.split(r',|&', lyrics_text)]

    #Year
    year = list(filter(lambda x: 'Year' in x, information_collection))[0].strip().split(':')[1]

    #Language
    language = list(filter(lambda x: 'Language' in x, information_collection))[0].strip().split(':')[1]

    #like & dislike
    rating_el = article.find('div', class_='rating')
    like = rating_el.find('button', id = 'like').find('span', id = 'lcnt').text
    dislike = rating_el.find('button', id = 'dlike').find('span', id = 'dcnt').text

    #movie songs
    track_table = article.find('table', id='tlist')
    track_trs = track_table.find_all('tr', attrs={'itemprop':'itemListElement'})

    track_info_tds = [track_tr.find('td') for track_tr in track_trs]

    track_infos = [{
        "Track_Name": track_info_td.find('a').text,
        "Track_Href": baseurl + track_info_td.find('a').get('href')
    } for track_info_td in track_info_tds]

    movie_detail = {
        "Name": title,
        "Casts": casts,
        "Music_Director": music_directors,
        "Director": directors,
        "Lyricist": lyric_writers,
        "Year": year.strip(),
        "Language": language.strip(),
        "Likes": like,
        "Dislikes": dislike,
        "Tracks": track_infos
    }
    return movie_detail

In [152]:
select_movie_query = 'SELECT * FROM MoviePage'
movie_page_res = cur.execute(select_movie_query)
movie_page_res_obj = movie_page_res.fetchone()
get_detail(movie_page_res_obj[2], movie_page_res_obj[1])

{"Name": "1 Pandu 4 Run 1 Wicket", "Casts": ["Vinai Krishna", "Hashika Dutt", "Sriman", "Senrayan"], "Music_Director": ["Karthik Nallamuthu"], "Director": ["Veera"], "Lyricist": ["Na. Muthukumar", "Snehan", "Kosimin"], "Year": "2014", "Language": "Tamil", "Likes": "1", "Dislikes": "2"}
[{'Track_Name': '1 Pandhu 4 Run 1 Wicket', 'Track_Href': 'https://tamilpaatu.com/1068/1-pandhu-4-run-1-wicket-mp3-song-1'}, {'Track_Name': 'Kalgal Rendum', 'Track_Href': 'https://tamilpaatu.com/1068/kalgal-rendum-mp3-song-1'}, {'Track_Name': 'Parallum Pandi', 'Track_Href': 'https://tamilpaatu.com/1068/parallum-pandi-mp3-song-1'}, {'Track_Name': 'Theme Music', 'Track_Href': 'https://tamilpaatu.com/1068/theme-music-mp3-song-1'}, {'Track_Name': 'Un Tholil', 'Track_Href': 'https://tamilpaatu.com/1068/un-tholil-mp3-song-1'}, {'Track_Name': 'Veesum Katril', 'Track_Href': 'https://tamilpaatu.com/1068/veesum-katril-mp3-song-1'}]
